<a href="https://colab.research.google.com/github/iampundir/Spark-Project/blob/master/Spark_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

a new user spends 20 seconds on the website and
we have to predict if this user will convert or not using the Logistic regression
line. We use the regression equation and try to predict the y value for
20 seconds time spent.

In [26]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!wget -q http://apache.osuosl.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz

#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [27]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

In [28]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext
import urllib.request

We then load and read the dataset within Spark using Dataframe. We have
to make sure we have opened the PySpark from the same directory folder
where the dataset is available or else we have to mention the directory path
of the data folder.

In [29]:
#import SparkSession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('log_reg').getOrCreate()

Now we will upload the required data file.

In [30]:
from google.colab import files
uploaded=files.upload()

Saving Logistic_data.csv to Logistic_data (1).csv


Lets check the file 

In [31]:
!ls

'Logistic_data (1).csv'   spark-2.4.6-bin-hadoop2.7
 Logistic_data.csv	  spark-2.4.6-bin-hadoop2.7.tgz
 sample_data		  spark-2.4.6-bin-hadoop2.7.tgz.1


Now we will load the file

In [32]:
#read the dataset
df=spark.read.csv("Logistic_data.csv",inferSchema=True,header=True)

In [33]:
from pyspark.sql.functions import *

**Exploratory Data Analysis**

In [34]:
#check the shape of the data 
print((df.count(),len(df.columns)))

(20000, 7)


the above output confirms the size of our dataset

we can then
validate the datatypes of the input values to check if we need to change/
cast any columns datatypes

In [35]:
#printSchema
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)



As we can see, there are two such columns (Country, Search_Engine),
which are categorical in nature and hence need to be converted into
numerical form.

In [36]:
#number of columns in dataset
df.columns

['_c0',
 'Country',
 'Age',
 'Repeat_Visitor',
 'Platform',
 'Web_pages_viewed',
 'Status']

Let’s have a look at the dataset using the show function in
Spark.

In [37]:
#view the dataset
df.show(5)

+---+---------+---+--------------+--------+----------------+------+
|_c0|  Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|
+---+---------+---+--------------+--------+----------------+------+
|  0|    India| 41|             1|   Yahoo|              21|     1|
|  1|   Brazil| 28|             1|   Yahoo|               5|     0|
|  2|   Brazil| 40|             0|  Google|               3|     0|
|  3|Indonesia| 31|             1|    Bing|              15|     1|
|  4| Malaysia| 32|             0|  Google|              15|     1|
+---+---------+---+--------------+--------+----------------+------+
only showing top 5 rows



We can now use the describe function to go over statistical measures of
the dataset.

In [38]:
#Exploratory Data Analysis
df.describe().show()

+-------+-----------------+--------+-----------------+-----------------+--------+-----------------+------------------+
|summary|              _c0| Country|              Age|   Repeat_Visitor|Platform| Web_pages_viewed|            Status|
+-------+-----------------+--------+-----------------+-----------------+--------+-----------------+------------------+
|  count|            20000|   20000|            20000|            20000|   20000|            20000|             20000|
|   mean|           9999.5|    null|         28.53955|           0.5029|    null|           9.5533|               0.5|
| stddev|5773.647027659381|    null|7.888912950773227|0.500004090187782|    null|6.073903499824976|0.5000125004687693|
|    min|                0|  Brazil|               17|                0|    Bing|                1|                 0|
|    max|            19999|Malaysia|              111|                1|   Yahoo|               29|                 1|
+-------+-----------------+--------+------------

We can observe that the average age of visitors is close to 28 years, and
they view around 9 web pages during the website visit.

Let us explore individual columns to understand the data in deeper
details. The groupBy function used along with counts returns the
frequency of each of the categories in the data.

In [39]:
df.groupBy('Country').count().show()

+---------+-----+
|  Country|count|
+---------+-----+
| Malaysia| 1218|
|    India| 4018|
|Indonesia|12178|
|   Brazil| 2586|
+---------+-----+



So, the maximum number of visitors are from Indonesia, followed by
India

In [40]:
df.groupBy('Platform').count().show()

+--------+-----+
|Platform|count|
+--------+-----+
|   Yahoo| 9859|
|    Bing| 4360|
|  Google| 5781|
+--------+-----+



The Yahoo search engine users are the highest in numbers

In [41]:
df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1|10000|
|     0|10000|
+------+-----+



We have an equal number of users who are converted and nonconverted.

Let’s use the groupBy function along with the mean to know more
about the dataset.

In [42]:
df.groupBy('Country').mean().show()

+---------+------------------+------------------+-------------------+---------------------+--------------------+
|  Country|          avg(_c0)|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|         avg(Status)|
+---------+------------------+------------------+-------------------+---------------------+--------------------+
| Malaysia| 9814.097701149425|27.792282430213465| 0.5730706075533661|   11.192118226600986|  0.6568144499178982|
|    India| 9998.998506719761|27.976854156296664| 0.5433051269288203|   10.727227476356397|  0.6212045793927327|
|Indonesia|10028.314419444901| 28.43159796354081| 0.5207751683363442|    9.985711939563148|  0.5422893742814913|
|   Brazil| 9951.910286156226|30.274168600154677|  0.322892498066512|    4.921113689095128|0.038669760247486466|
+---------+------------------+------------------+-------------------+---------------------+--------------------+



We have the highest conversion rate from Malaysia, followed by India.
The average number of web page visits is highest in Malaysia and lowest in
Brazil.

In [43]:
df.groupBy('Platform').mean().show()

+--------+------------------+------------------+-------------------+---------------------+------------------+
|Platform|          avg(_c0)|          avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|       avg(Status)|
+--------+------------------+------------------+-------------------+---------------------+------------------+
|   Yahoo|10029.549548635765|28.569226087838523| 0.5094837204584644|    9.599655137437875|0.5071508266558474|
|    Bing|10086.784403669724| 28.68394495412844| 0.4720183486238532|    9.114908256880733|0.4559633027522936|
|  Google| 9882.423629129908|28.380038055699707| 0.5149628092025601|    9.804878048780488|0.5210171250648676|
+--------+------------------+------------------+-------------------+---------------------+------------------+



We have the highest conversion rate from user visitors use the Google
search engine.

In [44]:
df.groupBy('Status').mean().show()

+------+---------+--------+-------------------+---------------------+-----------+
|Status| avg(_c0)|avg(Age)|avg(Repeat_Visitor)|avg(Web_pages_viewed)|avg(Status)|
+------+---------+--------+-------------------+---------------------+-----------+
|     1|10007.333| 26.5435|             0.7019|              14.5617|        1.0|
|     0| 9991.667| 30.5356|             0.3039|               4.5449|        0.0|
+------+---------+--------+-------------------+---------------------+-----------+



We can clearly see there is a strong connection between the conversion
status and the number of pages viewed along with repeat visits.

**Feature Engineering**

This is the part where we convert the categorical variable into numerical
form and create a single vector combining all the input features by using
Spark’s VectorAssembler.

In [45]:
#import required libraries

from pyspark.ml.feature import StringIndexer

Since we are dealing with two categorical columns, we will have to
convert the country and search engine columns into numerical form. The
machine learning model cannot understand categorical values.

The first step is to label the column using StringIndexer into
numerical form. It allocates unique values to each of the categories of the
column.

In [46]:
search_engine_indexer = StringIndexer(inputCol="Platform", outputCol="Search_Engine_Num").fit(df)
df = search_engine_indexer.transform(df)

In [47]:
df.show(3,False)

+---+-------+---+--------------+--------+----------------+------+-----------------+
|_c0|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Search_Engine_Num|
+---+-------+---+--------------+--------+----------------+------+-----------------+
|0  |India  |41 |1             |Yahoo   |21              |1     |0.0              |
|1  |Brazil |28 |1             |Yahoo   |5               |0     |0.0              |
|2  |Brazil |40 |0             |Google  |3               |0     |1.0              |
+---+-------+---+--------------+--------+----------------+------+-----------------+
only showing top 3 rows



The next step is to represent each of these values into the form of a
one hot encoded vector. However, this vector is a little different in terms of
representation as it captures the values and position of the values in the vector.

In [49]:
#one hot encoding
from pyspark.ml.feature import OneHotEncoder
search_engine_encoder = OneHotEncoder(inputCol="Search_Engine_Num", outputCol="Search_Engine_Vector")
df = search_engine_encoder.transform(df)

In [50]:
df.show(3,False)

+---+-------+---+--------------+--------+----------------+------+-----------------+--------------------+
|_c0|Country|Age|Repeat_Visitor|Platform|Web_pages_viewed|Status|Search_Engine_Num|Search_Engine_Vector|
+---+-------+---+--------------+--------+----------------+------+-----------------+--------------------+
|0  |India  |41 |1             |Yahoo   |21              |1     |0.0              |(2,[0],[1.0])       |
|1  |Brazil |28 |1             |Yahoo   |5               |0     |0.0              |(2,[0],[1.0])       |
|2  |Brazil |40 |0             |Google  |3               |0     |1.0              |(2,[1],[1.0])       |
+---+-------+---+--------------+--------+----------------+------+-----------------+--------------------+
only showing top 3 rows



In [51]:
df.groupBy('Platform').count().orderBy('count',ascending=False).show(5,False)

+--------+-----+
|Platform|count|
+--------+-----+
|Yahoo   |9859 |
|Google  |5781 |
|Bing    |4360 |
+--------+-----+



In [52]:
df.groupBy('Search_Engine_Num').count().orderBy('count',ascending=False).show(5,False)

+-----------------+-----+
|Search_Engine_Num|count|
+-----------------+-----+
|0.0              |9859 |
|1.0              |5781 |
|2.0              |4360 |
+-----------------+-----+



The final feature that we would be using for building Logistic
Regression is Search_Engine_Vector.

In [53]:
df.groupBy('Search_Engine_Vector').count().orderBy('count',ascending=False).show(5,False)

+--------------------+-----+
|Search_Engine_Vector|count|
+--------------------+-----+
|(2,[0],[1.0])       |9859 |
|(2,[1],[1.0])       |5781 |
|(2,[],[])           |4360 |
+--------------------+-----+



 Let’s understand what these column
values represent.

 (2,[0],[1.0]) represents a vector of length 2 , with 1 value :

Size of Vector – 2

Value contained in vector – 1.0

Position of 1.0 value in vector – 0th place

This kind of representation allows the saving of computational space
and hence a faster time to compute. The length of the vector is equal to
one less than the total number of elements since each value can be easily
represented with just the help of two columns.

Let’s repeat the same procedure for the other categorical column
(Country).

In [54]:
country_indexer = StringIndexer(inputCol="Country", outputCol="Country_Num").fit(df)
df = country_indexer.transform(df)

In [55]:
df.select(['Country','Country_Num']).show(3,False)

+-------+-----------+
|Country|Country_Num|
+-------+-----------+
|India  |1.0        |
|Brazil |2.0        |
|Brazil |2.0        |
+-------+-----------+
only showing top 3 rows



In [56]:
#one hot encoding
country_encoder = OneHotEncoder(inputCol="Country_Num", outputCol="Country_Vector")
df = country_encoder.transform(df)

In [57]:
df.select(['Country','country_Num','Country_Vector']).show(3,False)

+-------+-----------+--------------+
|Country|country_Num|Country_Vector|
+-------+-----------+--------------+
|India  |1.0        |(3,[1],[1.0]) |
|Brazil |2.0        |(3,[2],[1.0]) |
|Brazil |2.0        |(3,[2],[1.0]) |
+-------+-----------+--------------+
only showing top 3 rows



In [58]:
df.groupBy('Country').count().orderBy('count',ascending=False).show(5,False)

+---------+-----+
|Country  |count|
+---------+-----+
|Indonesia|12178|
|India    |4018 |
|Brazil   |2586 |
|Malaysia |1218 |
+---------+-----+



In [59]:
df.groupBy('Country_Num').count().orderBy('count',ascending=False).show(5,False)

+-----------+-----+
|Country_Num|count|
+-----------+-----+
|0.0        |12178|
|1.0        |4018 |
|2.0        |2586 |
|3.0        |1218 |
+-----------+-----+



In [60]:
df.groupBy('Country_Vector').count().orderBy('count',ascending=False).show(5,False)

+--------------+-----+
|Country_Vector|count|
+--------------+-----+
|(3,[0],[1.0]) |12178|
|(3,[1],[1.0]) |4018 |
|(3,[2],[1.0]) |2586 |
|(3,[],[])     |1218 |
+--------------+-----+



Now that we have converted both the categorical columns into
numerical forms, we need to assemble all of the input columns into a
single vector that would act as the input feature for the model.

So, we select the input columns that we need to use to create the single
feature vector and name the output vector as features.

In [61]:
from pyspark.ml.feature import VectorAssembler

In [64]:
df_assembler = VectorAssembler(inputCols=['Search_Engine_Vector','Country_Vector','Age', 'Repeat_Visitor','Web_pages_viewed'], outputCol="features")
df = df_assembler.transform(df)

In [65]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Repeat_Visitor: integer (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Web_pages_viewed: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- Search_Engine_Num: double (nullable = false)
 |-- Search_Engine_Vector: vector (nullable = true)
 |-- Country_Num: double (nullable = false)
 |-- Country_Vector: vector (nullable = true)
 |-- features: vector (nullable = true)



As we can see, now we have one extra column named features, which
is nothing but a combination of all the input features represented as a
single dense vector

In [66]:
df.select(['features','Status']).show(10,False)

+-----------------------------------+------+
|features                           |Status|
+-----------------------------------+------+
|[1.0,0.0,0.0,1.0,0.0,41.0,1.0,21.0]|1     |
|[1.0,0.0,0.0,0.0,1.0,28.0,1.0,5.0] |0     |
|(8,[1,4,5,7],[1.0,1.0,40.0,3.0])   |0     |
|(8,[2,5,6,7],[1.0,31.0,1.0,15.0])  |1     |
|(8,[1,5,7],[1.0,32.0,15.0])        |1     |
|(8,[1,4,5,7],[1.0,1.0,32.0,3.0])   |0     |
|(8,[1,4,5,7],[1.0,1.0,32.0,6.0])   |0     |
|(8,[1,2,5,7],[1.0,1.0,27.0,9.0])   |0     |
|(8,[0,2,5,7],[1.0,1.0,32.0,2.0])   |0     |
|(8,[2,5,6,7],[1.0,31.0,1.0,16.0])  |1     |
+-----------------------------------+------+
only showing top 10 rows



Let us select only features column as input and the Status column as
output for training the logistic regression model.

In [67]:
#select data for building model
model_df=df.select(['features','Status'])

**Splitting the Dataset**

We have to split the dataset into a training and test dataset in order to train
and evaluate the performance of the logistic regression model. We split it
in a 75/25 ratio and train our model on 75% of the dataset. Another use of
splitting the data is that we can use 75% of the data to apply cross-validation
in order to come up with the best Hyperparameters. Cross-validation can be
of a different type where one part of the training data is kept for training and
the remaining part is used for validation purposes. K-fold cross-validation is
primarily used to train the model with the best Hyperparameters.

We can print the shape of train and test data to validate the size

In [68]:
from pyspark.ml.classification import LogisticRegression

In [69]:
#split the data 
training_df,test_df=model_df.randomSplit([0.75,0.25])

In [70]:
training_df.count()

14994

In [71]:
training_df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 7476|
|     0| 7518|
+------+-----+



In [72]:
test_df.count()

5006

In [73]:
test_df.groupBy('Status').count().show()

+------+-----+
|Status|count|
+------+-----+
|     1| 2524|
|     0| 2482|
+------+-----+



**Build and Train Logistic Regression Model**

Now,we build and train the logistic regression model using features
as the input column and status as the output column.

In [74]:
 from pyspark.ml.classification import LogisticRegression

In [75]:
log_reg=LogisticRegression(labelCol='Status').fit(training_df)

**Training Results**

We can access the predictions made by the model using the evaluate
function in Spark that executes all the steps in an optimized way. That
gives another Dataframe that contains four columns in total, including
prediction and probability. The prediction column signifies the class
label that the model has predicted for the given row and probability
column contains two probabilities (probability for negative class at 0th
index and probability for positive class at 1st index

In [76]:
#Training Results
train_results=log_reg.evaluate(training_df).predictions
train_results.filter(train_results['Status']==1).filter(train_results['prediction']==1).select(['Status','prediction','probability']).show(10,False)

+------+----------+----------------------------------------+
|Status|prediction|probability                             |
+------+----------+----------------------------------------+
|1     |1.0       |[0.3075148055014183,0.6924851944985817] |
|1     |1.0       |[0.3075148055014183,0.6924851944985817] |
|1     |1.0       |[0.3075148055014183,0.6924851944985817] |
|1     |1.0       |[0.1717327850840228,0.8282672149159772] |
|1     |1.0       |[0.1717327850840228,0.8282672149159772] |
|1     |1.0       |[0.1717327850840228,0.8282672149159772] |
|1     |1.0       |[0.1717327850840228,0.8282672149159772] |
|1     |1.0       |[0.08826315592210453,0.9117368440778955]|
|1     |1.0       |[0.08826315592210453,0.9117368440778955]|
|1     |1.0       |[0.08826315592210453,0.9117368440778955]|
+------+----------+----------------------------------------+
only showing top 10 rows



Probability at 0 index is for 0 class and probabilty as 1 index is for 1 class

So, in the above results, probability at the 0th index is for Status = 0
and probability as 1st index is for Status =1.

In [77]:
correct_preds=train_results.filter(train_results['Status']==1).filter(train_results['prediction']==1).count()

In [78]:

training_df.filter(training_df['Status']==1).count()

7476

In [79]:
#accuracy on training dataset 
float(correct_preds)/(training_df.filter(training_df['Status']==1).count())

0.9394060995184591

We can filter the columns that we want to see using the select keyword.

In [80]:
#Test Set results
results=log_reg.evaluate(test_df).predictions
results.select(['Status','prediction']).show(10,False)

+------+----------+
|Status|prediction|
+------+----------+
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|0     |0.0       |
|1     |0.0       |
|0     |0.0       |
+------+----------+
only showing top 10 rows



In [81]:
results.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Status: integer (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



Since this is a classification problem, we will use a confusion matrix to
gauge the performance of the model.

**Confusion Matrix**

We will manually create the variables for true positives, true negatives,
false positives, and false negatives to understand them better rather than
using the direct inbuilt function.

In [82]:
#confusion matrix
true_postives = results[(results.Status == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.Status == 0) & (results.prediction == 0)].count()
false_positives = results[(results.Status == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.Status == 1) & (results.prediction == 0)].count()

In [83]:
print (true_postives)
print (true_negatives)
print (false_positives)
print (false_negatives)
print(true_postives+true_negatives+false_positives+false_negatives)
print (results.count())

2349
2338
144
175
5006
5006


**Accuracy**

accuracy is the most basic metric
for evaluating any classifier; however, this is not the right indicator of
the performance of the model due to dependency on the target class
balance.

In [85]:
accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)

0.9362764682381143


The accuracy of the model that we have built is around 94%.

**Recall**

Recall rate shows how much of the positive class cases we are able to
predict correctly out of the total positive class observations.

In [84]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

0.9306656101426307


The recall rate of the model is around 93%

Precision

Precision rate talks about the number of true positives predicted
correctly out of all the predicted positives observations:

In [86]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.9422382671480144


So, the recall rate and precision rate are also in the same range, which
is due to the fact that our target class was well balanced.